In [16]:
# Import libraries
import pandas as pd
import numpy as np

In [17]:
# Load csv to a Dataframe
df = pd.read_csv('output_data\\rating_blockbusters.csv')

# Display number of rated movies
print(df['movieId'].nunique())

df.head(2)

1340


,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5


In [18]:
# Load csv to a Dataframe
movie_df = pd.read_csv('output_data\\movie_2000.csv')

#### Pivoting the DataFrame into a Factorization Matrix

In [19]:
# Splitting the DataFrame into 10 equal chunks to lighten Pandas work load
user_splits = np.array_split(df['userId'].unique(), 10)

# Create empty list
df_pivot_list = []

# For loop to pivot table
for split in user_splits:
    df_subset = df[df['userId'].isin(split)]
    df_pivot_subset = pd.pivot_table(df_subset, values='rating', index='userId', columns='movieId')
    df_pivot_list.append(df_pivot_subset)
    print("New split on duty !")

# Concatenate the 10 chunks
df_pivot = pd.concat(df_pivot_list)

print('Job Done: Factorization Matrix Ready')


New split on duty !
New split on duty !
New split on duty !
New split on duty !
New split on duty !
New split on duty !
New split on duty !
New split on duty !
New split on duty !
New split on duty !
Job Done: Factorization Matrix Ready


In [26]:
# Display matrix shape
df_pivot.shape

(137877, 1340)

In [21]:
# Show the first rows of the matrix
df_pivot.head(2)

movieId,1,2,3,5,6,7,9,10,11,12,...,79702,80463,81591,81834,81845,87232,88125,89745,91529,99114
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Recommendation with genres

In [30]:
# Load csv to a Dataframe
movie_df = pd.read_csv('output_data\\movie_blockbusters.csv')

# Set 'movieId' as the index of the DataFrame
movie_df.set_index('movieId', inplace = True)
movie_df['year'] = movie_df['year'].astype(str)
print(movie_df.shape)
movie_df.head(2)

(1377, 22)


,title,genres,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Jumanji,Adventure|Children|Fantasy,1995,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Creating Dataframe with numeric types only columns

In [31]:
movie_df_num = movie_df.select_dtypes(include=[float, int])
movie_df_num.head(2)

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,
1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


#### Create Pearson Correlation Function

In [32]:
import warnings

# Ignore warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


def matching_genres(movie_title):
    # Find index of target movie
    target_movie_index = int(movie_df.index[movie_df['title'] == movie_title][0])
    
    # Select target movie genres
    target_genres = movie_df_num.loc[target_movie_index]

    # Calculate correlation score of target movie genres
    correlations = movie_df_num.apply(lambda row: row.corr(target_genres), axis=1)
    
    # Create DataFrame with genres correlation scores
    df_genre = pd.DataFrame(correlations, columns=['PearsonG'])
    
    # Add titles, genres and year
    df_genre = df_genre.join(movie_df[['title', 'genres', 'year']])
    
    # Sort score in descending order
    df_genre = df_genre.sort_values('PearsonG', ascending=False)
    df_genre.dropna(inplace=True)

    # Calculate correlation score with movie ratings
    target = df_pivot[target_movie_index]
    similar_to_target = df_pivot.corrwith(target)
    
    # Create DataFrame with movie correlation scores
    corr_target = pd.DataFrame(similar_to_target, columns=['PearsonR'])
    
    # Drop NaN
    corr_target.dropna(inplace=True)

    # Sort score in descending order
    corr_target = corr_target.sort_values('PearsonR', ascending=False)
    
    # Cast index in int type
    corr_target.index = corr_target.index.map(int)
    
    # Join genres and ratings correlation DataFrames
    df_combined = corr_target.join(df_genre.set_index(df_genre.index), how='inner')
    
    # Sort descending PearsonR et PearsonG columns
    df_combined = df_combined.sort_values(['PearsonR', 'PearsonG'], ascending=False)
    
    # Display results
    print("Because you watch this movie :", movie_title)
    print("Our Top10 Selection of 2000's movies for you !")
    print(df_combined.head(10))#.to_string(index=False))

# Call function
#matching_genres(matching_title)


#### Call it randomly with a function to generate recommandation

In [34]:
import random

def random_movie_recommendation():
    # Randomly select a movie title from the movie_df['title'] column
    random_title = random.choice(movie_df['title'].tolist())
    
    # Print the selected title (optional)
    print(f"Randomly selected movie: '{random_title}'")
    
    # Call the matching_genres function with the randomly selected title
    matching_genres(random_title)

# Call the function to get a random movie recommendation
random_movie_recommendation()


Randomly selected movie: 'Transformers'
Because you watch this movie : Transformers
Our Top10 Selection of 2000's movies for you !
         PearsonR  PearsonG                                           title  \
movieId                                                                       
53996    1.000000  1.000000                                    Transformers   
6383     0.507701  0.484481  2 Fast 2 Furious (Fast and the Furious 2, The)   
145      0.491382  0.277746                                        Bad Boys   
77561    0.484679  0.864099                                      Iron Man 2   
7454     0.479523  0.050000                                     Van Helsing   
34150    0.478691  0.484481                                  Fantastic Four   
8644     0.472626  0.683333                                        I, Robot   
59315    0.467138  0.484481                                        Iron Man   
8810     0.458459  0.683333                         AVP: Alien vs. Predator   


#### Call it manually to generate recommandation

In [ ]:
matching_genres('mammuth')

Because you watch this movie : mammuth
Our Top10 Selection of 2000's movies for you !
         PearsonR  PearsonG                              title        genres  \
movieId                                                                        
65230         1.0       1.0                         marley  me  Comedy|Drama   
66509         1.0       1.0                       funny people  Comedy|Drama   
8366          1.0       1.0                              saved  Comedy|Drama   
7460          1.0       1.0              coffee and cigarettes  Comedy|Drama   
71466         1.0       1.0                        city island  Comedy|Drama   
5890          1.0       1.0                             elling  Comedy|Drama   
78116         1.0       1.0                        please give  Comedy|Drama   
6620          1.0       1.0                  american splendor  Comedy|Drama   
6339          1.0       1.0  man on the train homme du train l  Comedy|Drama   
55280         1.0       1.0       